In [47]:
# predict_new_data_with_loaded_model (LSTM)

import numpy as np
import pandas as pd
from os.path import isfile, join
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras

def create_sequence(data, window_size):
    sequences = []
    targets = []
    for i in range(len(data) - window_size):
        sequences.append(data[i:i+window_size])
        targets.append(data[i+window_size])
    return np.array(sequences), np.array(targets)


def prepare_data(new_data, window_size, target_col=None, scaler=MinMaxScaler):
  if target_col is not None:
    # Create a target variable
    new_data['Target'] = new_data[target_col].shift(-1)

    # Drop the last row with NaN target
    new_data.dropna(inplace=True)

    # Drop target_col from data
    new_data = new_data.drop(target_col,axis=1)

  # Scaling
  scaler = scaler()
  new_data_scaled = scaler.fit_transform(new_data)

  # Create sequence of train and test sets
  new_data_X, new_data_y = create_sequence(new_data_scaled, window_size)

  return new_data_X, new_data_y

def predict_new_data(model_file_names, new_data, window_size, target_col=None, scaler=MinMaxScaler(), model_directory=None):
    results = []
    
    for model_file_name in model_file_names:
        if model_directory is not None:
            model_file_path = join(model_directory, model_file_name)
        else:
            model_file_path = model_file_name

        if isfile(model_file_path):   
            try:
                best_model = load_model(model_file_path)
            except Exception as e:
                print(f"Error occurred while loading the model: {e}")
                continue
        else:
            print(f"Model file path {model_file_path} doesn't exist.")
            continue

        # Prepare data
        new_data_X, new_data_y = prepare_data(new_data, window_size=window_size, target_col=target_col, scaler=scaler) 

        input_shape = (window_size, new_data_X.shape[2])
         
        new_data_y_pred = best_model.predict(new_data_X)

        if target_col is not None:
            # Reshape y_test, y_pred, X_test, and X_train to be 2D arrays
            new_data_y_2d = new_data_y[:, -1]
            new_data_y_pred_2d = new_data_y_pred[:, -1]
            new_data_X_2d = new_data_X.reshape(-1, new_data_X.shape[2])

        else:
            # Reshape y_test, y_pred, X_test, and X_train to be 2D arrays
            new_data_y_2d = new_data_y.reshape(-1, 1)
            new_data_y_pred_2d = new_data_y_pred.reshape(-1, 1)
            new_data_X_2d = new_data_X.reshape(-1, new_data_X.shape[2])

        # Calculate metrics
        new_rmse = np.sqrt(mean_squared_error(new_data_y_2d, new_data_y_pred_2d))
        new_r2 = r2_score(new_data_y_2d, new_data_y_pred_2d)

        # Create a dictionary with results
        result = {
            "Model": model_file_name,
            "New data R²": round(new_r2, 3),
            "New data RMSE": round(new_rmse, 3),
        }
        results.append(result)

    # Create a dataframe with results
    results_df = pd.DataFrame(results)
    return results_df


In [58]:
# predict_new_data_with_loaded_model (deep learning model)
def predict_new_data(model_file_names, new_data, window_size, target_col=None, scaler=MinMaxScaler(), model_directory=None):
    results = []
    
    for model_file_name in model_file_names:
        if model_directory is not None:
            model_file_path = join(model_directory, model_file_name)
        else:
            model_file_path = model_file_name

        if isfile(model_file_path):   
            try:
                best_model = load_model(model_file_path)
            except Exception as e:
                print(f"Error occurred while loading the model: {e}")
                continue
        else:
            print(f"Model file path {model_file_path} doesn't exist.")
            continue

        # Prepare data
        new_data_X, new_data_y = prepare_data(new_data, window_size=window_size, target_col=target_col, scaler=scaler) 

        input_shape = (window_size, new_data_X.shape[2])
         
        new_data_y_pred = best_model.predict(new_data_X)

        if target_col is not None:
            # Reshape y_test, y_pred, X_test, and X_train to be 2D arrays
            new_data_y_2d = new_data_y[:, -1]
            new_data_y_pred_2d = new_data_y_pred[:, -1]
            new_data_X_2d = new_data_X.reshape(-1, new_data_X.shape[2])

        else:
            # Reshape y_test, y_pred, X_test, and X_train to be 2D arrays
            new_data_y_2d = new_data_y.reshape(-1, 1)
            new_data_y_pred_2d = new_data_y_pred.reshape(-1, 1)
            new_data_X_2d = new_data_X.reshape(-1, new_data_X.shape[2])

        # Calculate metrics
        new_rmse = np.sqrt(mean_squared_error(new_data_y_2d, new_data_y_pred_2d))
        new_r2 = r2_score(new_data_y_2d, new_data_y_pred_2d)

        # Create a dictionary with results
        result = {
            "Model": model_file_name,
            "Train R²": round(new_r2, 3),
            "Train RMSE": round(new_rmse, 3),
        }
        results.append(result)

    # Create a dataframe with results
    results_df = pd.DataFrame(results)
    return results_df


In [59]:
import yfinance as yf
# Load the data from Yahoo Finance
symbol = 'AAPL'
start_date = '2015-01-01'
end_date = '2016-07-12'
new_data = yf.download(symbol, start=start_date, end=end_date)
new_data = new_data.drop('Adj Close', axis=1)

[*********************100%***********************]  1 of 1 completed


In [61]:
model_file_name = ['final_model0.h5', 'final_model1.h5', 'final_model2.h5', 'final_model3.h5', 
                   'final_model4.h5', 'final_model5.h5', 'final_model6.h5', 'final_model7.h5',
                   'final_model8.h5', 'final_model9.h5']
predict_new_data(model_file_name, new_data, window_size=10, target_col=None, scaler=MinMaxScaler, model_directory=None)

X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/12 [==============================] - 0s 2ms/step
X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/12 [==============================] - 0s 2ms/step
X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/12 [==============================] - 0s 2ms/step
X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/12 [==============================] - 0s 2ms/step
X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/12 [==============================] - 0s 2ms/step
X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/12 [==============================] - 0s 2ms/step
X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/12 [==============================] - 0s 2ms/step
X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/12 [==============================] - 0s 2ms/step
X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/12 [==============================] - 0s 2ms/step
X_train shape: (373, 10, 5)
y_train shape: (373, 5)
12/

,Model,Train R²,Train RMSE
0,final_model0.h5,0.133,0.267
1,final_model1.h5,0.236,0.250
2,final_model2.h5,0.150,0.264
3,final_model3.h5,0.311,0.238
4,final_model4.h5,0.133,0.267
5,final_model5.h5,0.144,0.265
6,final_model6.h5,0.118,0.269
7,final_model7.h5,0.105,0.271
8,final_model8.h5,0.086,0.274
9,final_model9.h5,0.074,0.276
